This demo run successful in goolge colab for person detection, for time reason, only run 20 epochs.


download code!

In [1]:
!git clone https://github.com/yl305237731/flexible-yolov5.git

Cloning into 'flexible-yolov5'...
remote: Enumerating objects: 481, done.
remote: Counting objects: 100% (481/481), done.
remote: Compressing objects: 100% (341/341), done.
remote: Total 481 (delta 249), reused 323 (delta 115), pack-reused 0
Receiving objects: 100% (481/481), 1.44 MiB | 29.41 MiB/s, done.
Resolving deltas: 100% (249/249), done.


download dataset!

In [2]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar

!tar -xf VOCtrainval_11-May-2012.tar

--2021-07-20 07:31:08--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/x-tar]
Saving to: ‘VOCtrainval_11-May-2012.tar’

VOCtrainval_11-May- 100%[===================>]   1.86G  14.9MB/s    in 2m 8s   

2021-07-20 07:33:16 (14.9 MB/s) - ‘VOCtrainval_11-May-2012.tar’ saved [1999639040/1999639040]



install python requirements

In [16]:
!pip install -r flexible-yolov5/requirements.txt

Make data for yolov5 format.
I only training for person detection.

This script in od/data/transform_voc.py

In [25]:
import xml.etree.ElementTree as ET
import os
import shutil

global year
year = '2012'


def convert(size, box):
    dw = 1. / (size[0])
    dh = 1. / (size[1])
    x = max((box[0] + box[1]) / 2.0 - 1, 0)
    y = max((box[2] + box[3]) / 2.0 - 1, 0)
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)


def convert_annotation(data_dir, image_id, train, classes):
    in_file = open(os.path.join(data_dir, 'VOC' + year + '/Annotations/%s.xml' % (image_id)), encoding='utf-8')
    if train:
        out_file = open(os.path.join(data_dir, 'labels/train/%s.txt' % (image_id)), 'w', encoding='utf-8')
    else:
        out_file = open(os.path.join(data_dir, 'labels/val/%s.txt' % (image_id)), 'w', encoding='utf-8')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes:
          continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text),
             float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')


def transform_voc(data_dir, classes, c_year=None):
    if c_year is not None:
        year = c_year

    if not os.path.exists(os.path.join(data_dir, 'images')):
        os.makedirs(os.path.join(data_dir, 'images/train'))
        os.makedirs(os.path.join(data_dir, 'images/val'))
    if not os.path.exists(os.path.join(data_dir, 'labels')):
        os.makedirs(os.path.join(data_dir, 'labels/train'))
        os.makedirs(os.path.join(data_dir, 'labels/val'))

    # make train labels
    train_image_ids = open(os.path.join(data_dir, 'VOC' + year + '/ImageSets/Main/train.txt'), encoding='utf-8')
    for image_id in train_image_ids:
        image_id = image_id.strip()
        convert_annotation(data_dir, image_id, True, classes)
        img_path = os.path.join(data_dir, 'VOC' + year + "/JPEGImages", image_id + '.jpg')
        shutil.copy(img_path, os.path.join(data_dir, 'images/train/'))

    # make val labels
    val_image_ids = open(os.path.join(data_dir, 'VOC' + year + '/ImageSets/Main/val.txt'), encoding='utf-8')
    for image_id in val_image_ids:
        image_id = image_id.strip()
        convert_annotation(data_dir, image_id, False, classes)
        img_path = os.path.join(data_dir, 'VOC' + year + "/JPEGImages", image_id + '.jpg')
        shutil.copy(img_path, os.path.join(data_dir, 'images/val/'))

    return os.path.join(data_dir, 'images/train'), os.path.join(data_dir, 'images/val')


data_dir = './VOCdevkit'
classes = ['person'] # detection class
transform_voc(data_dir, classes, year)


('./VOCdevkit/images/train', './VOCdevkit/images/val')

Change configs/data.yaml

1. set your dataset path
2. change number of classes
3. set names, for visualization.

In [ ]:
# Custom data for person detection, configs/data/yaml


# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: /content/VOCdevkit/images/train/
val: /content/VOCdevkit/images/val/

# number of classes
nc: 1

# class names
names: ['person']


Here we use yolov5s, change the nc to your number of classes. only one class, set nc=1. configs/model_yolo.yaml


In [ ]:
backbone:
  type: YOLOv5
  version: s # x, m, l ,x
  focus: True
  with_C3TR: False
head:
  nc: 1
  stride: [8.0, 16.0, 32.0]
  anchors:
    - [10,13, 16,30, 33,23]  # P3/8
    - [30,61, 62,45, 59,119]  # P4/16
    - [116,90, 156,198, 373,326]  # P5/32

Start training and use default parameters.

In [ ]:
!cd flexible-yolov5/ && python scripts/train.py

fatal: ambiguous argument 'main..origin/master': unknown revision or path not in the working tree.
Use '--' to separate paths from revisions, like this:
'git <command> [<revision>...] -- [<file>...]'
github: Command 'git rev-list main..origin/master --count' returned non-zero exit status 128.
YOLOv5 2e0b9bf torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, batch_size=32, bucket='', cache_images=False, cfg='configs/model_yolo.yaml', data='configs/data.yaml', device='', epochs=300, evolve=False, exist_ok=False, global_rank=-1, hyp='configs/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp6', single_cls=False, sync_bn=False, total_batch_size=32, weights='', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/